# Hackathon RIIAA 2021 "JusticIA para los desaparecidos"


Dedicado a todos los que desaparecieron de nuestra vista, pero no de nuestra memoría. Y también para todos los hijos, estudiantes, padres y el pueblo de México; que los sigue y seguirá buscando hasta tener Justicia y Paz. 

Este Notebook fué realizado por:

*   Steve Alejandro Avedaño García (https://github.com/steve2ag)
*   Edgar Alberto Arellano Ruelas (https://github.com/edx04)
*   Paola Bonifacio Ramírez (https://github.com/Pao16)
*   Juan Carlos Bonifacio Ramírez (https://github.com/JCjc77)
*   David Augusto Galván Alvarado (https://github.com/IngeniiHub)

**Funcionamiento:** analíza imágenes de ficheros con personas desparecidas del periodo llamado "la guerra sucia". Extrae texto con información clave para ayudar a su busqueda y recuperación.

Para un tutorial detallado de cómo usarlo, recomiendamos [visitar este artículo].

**"...¿A quién le hacen falta los desaparecidos? ...** le hacen falta a sus hijos, padres, estudiantes y sobre todo al pueblo que merece conocer sus raices que han tratado de desaparecer, merecemos saber la verdad..."

                                   - Juan Carlos Mendoza Comité ¡Eureka!



#Licencia e inicialización



In [1]:
# @title Licensed under the MIT License

# Copyright (c) 2021 by:
# David Augusto Galván Alvarado
# Juan Carlos Bonifacio Ramírez 
# Paola Bonifacio Ramírez 
# Edgar Alberto Arellano Ruelas 
# Steve Alejandro Avedaño García 



# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [2]:
#Instalación y actualización de librerías (puede tardar)
!pip install --upgrade google-cloud-vision      
!pip install --upgrade pip                      
!pip install flair                              

In [3]:
#importamos librerías de phyton
import pandas as pd
import numpy as np

# Extracción de Datos del Hackathon

In [4]:
#Montar Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Cargar las fotografías a variables respectivas
from PIL import Image
from glob import glob
from random import sample

in_dir = 'drive/MyDrive/JUSTICIIA/Datos - Hackathon JusticIA/'    #Ruta del drive con la carpeta
files_auto = glob(in_dir+'/Fichas_auto/*.*')       #archivos depurados automáticamente (calidad media)
files_manual = glob(in_dir+'/Fichas_manual/*.*')     #archivos depurados manualmente (calidad alta)
files_prueba = glob(in_dir+'/Evaluacion/Reto2/*.*')   #archivos del conjunto de prueba

# Conexión con entorno de Google Cloud (Key req)

In [6]:
#Se requiere cargar una cuenta de servicio de google cloud [Archivo JSON]
#tutorial para usar y obtener JSON :  https://cloud.google.com/vision/docs/before-you-begin


from google.colab import files
uploaded = files.upload()
import os
name = ""

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
  name =fn  

Saving ocrhack-323621-e49ca6650f3f (1).json to ocrhack-323621-e49ca6650f3f (1) (1).json
User uploaded file "ocrhack-323621-e49ca6650f3f (1).json" with length 2311 bytes


In [7]:
import os

# Se carga la variable de entorno :
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = name
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

Service Account Key: ocrhack-323621-e49ca6650f3f (1).json


# Extracción de texto

In [8]:
# ORC de google cloud
# Funcion para extraer el texto de las imagenes
def detect_document(path):
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.document_text_detection(image=image)
    return response.text_annotations[0].description

In [9]:
#Funcion para generar el dataframe con los nombres de los archivos y sus respectivos textos obtenidos del OCR

import re
names = []
texts = []
def generate_dataframe(files):
  for file in files:
    name = re.search("[\w,-]*\.JPG",file).group(0)
    print(name)
    names.append(name)
    texts.append(detect_document(file))

In [10]:
#Para llamar a la funcion se le tiene que pasar una lista de direciones 

generate_dataframe(files_prueba)

Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel_Olea_Castaneyra_Rafael,_ACNR-19-.JPG
Ficheros_Detenidos_Desaparecidos_Morales_Lopez_Delia_Morales_Lopez_Delia-23-.JPG
Ficheros_UAG_76-11-16_a_77-03-19_UAG,_fichas-113-.JPG
Ficheros_UAG_73-10-29_a_73-12-07_UAG,_fichas-134-.JPG
Ficheros_Brigada_Campesina_de_Ajusticiamiento_BCA_Brigada_Campesina_de_Ajusticiamiento-79-.JPG
Ficheros_Detenidos_Desaparecidos_Perez_Lopez_Martha_Perez_Lopez_Martha-21-.JPG
Ficheros_FAR,_concentrado_general_FAR,_concentrado_general-157-.JPG
Ficheros_PPUA_Fichas_PPUA,_Fichas-98-.JPG
Ficheros_Represores_Martinez_Adame_Arturo_Martinez_Adame_Arturo-31-.JPG
Ficheros_Union_Campesina_Independiente,_fichas_UCI,_fichas-2-.JPG


In [12]:
#generación de dataframe
df = pd.DataFrame(data={"nombre":names,
                  "texto":texts})

In [13]:
import pandas as pd
import re

In [14]:
#Funcion para recuperar el expediente

def obtener_expediente(x):
  res = re.search("(\s[Ee]?[Xx][Pp][\=|\.]?[\s]?[\d\.\-\+]{5,})|(\s[Ee][Xx][Pp]?[\=|\.]?[\s]?[\d\-\.\+]{5,})|(\s[Ee][Xx]?[Pp][\=|\.]?[\s]?[\d\.\-\+]{5,})|([\d]{2,3}-[\d]{2,3}-[\d]{2,3})|(\s[Ee][Xx][Pp].\s?[\d\.\-\+]{5,})",x)

  if res:
    return res.group(0)
  else:
    return ""
df['exp'] = df['texto'].apply(lambda x: obtener_expediente(x))

# Prueba y mejoras en la extracción

In [18]:
#Prueba de extracción simple
df['texto'][0]

'OLEA CASTANEYRA Rafael Dr. Exp-11-240-74\n-9-\nH-98 L-5\nde mayo de 1965 siendo medico interno del Hospi-\ntal Civil de Oaxaca, firmo entre otros un docu-\nmento entregado al Direcctor del ospital comu-\nnicaban su decisión de ir al paro en apoyo a la\nAsociación Mexicana de Médicos Residentes e Inte\nnos.-Realizó sus estudios profesionales en la -\nEscuela Superior de Medicina del I.P.N..-Por -\nconducto de su esposa conoció a la Prof. CONCCP=\nCION SOLIS MORALES, quien en 1969 le presento a\nFAUSTO AVILA, del grupo de Genaro Vazquez Rojas\ny dias despues de ser secuetrado DONACIANO LUNA\nRADILLA, a quien liberaron en los primeros dias\ndel mes de enero de 1971 mediante un rescate de\n$500,000.00 llegaron a su domicilio la Prof. SO=\nLIS MORALES Y SÁNTON MENDEZ BAILON llevandose\n'

In [19]:
# Mejora 1: con función para unir palabras separadas por salto de linea 
#  Ejemplo:  por Consti-/ntución


df['texto2'] = df['texto'].apply(lambda x:re.sub("-\n","",x))

In [20]:
#Prueba con mejora 1
df['texto2'][0]

'OLEA CASTANEYRA Rafael Dr. Exp-11-240-74\n-9H-98 L-5\nde mayo de 1965 siendo medico interno del Hospital Civil de Oaxaca, firmo entre otros un documento entregado al Direcctor del ospital comunicaban su decisión de ir al paro en apoyo a la\nAsociación Mexicana de Médicos Residentes e Inte\nnos.-Realizó sus estudios profesionales en la Escuela Superior de Medicina del I.P.N..-Por conducto de su esposa conoció a la Prof. CONCCP=\nCION SOLIS MORALES, quien en 1969 le presento a\nFAUSTO AVILA, del grupo de Genaro Vazquez Rojas\ny dias despues de ser secuetrado DONACIANO LUNA\nRADILLA, a quien liberaron en los primeros dias\ndel mes de enero de 1971 mediante un rescate de\n$500,000.00 llegaron a su domicilio la Prof. SO=\nLIS MORALES Y SÁNTON MENDEZ BAILON llevandose\n'

In [21]:
# Mejora 2: Se sustituyen todos los saltos de linea restantes por espacios
df['texto_f'] = df['texto2'].apply(lambda x:re.sub("\n"," ",x))

In [22]:
df['texto_f'][0]

'OLEA CASTANEYRA Rafael Dr. Exp-11-240-74 -9H-98 L-5 de mayo de 1965 siendo medico interno del Hospital Civil de Oaxaca, firmo entre otros un documento entregado al Direcctor del ospital comunicaban su decisión de ir al paro en apoyo a la Asociación Mexicana de Médicos Residentes e Inte nos.-Realizó sus estudios profesionales en la Escuela Superior de Medicina del I.P.N..-Por conducto de su esposa conoció a la Prof. CONCCP= CION SOLIS MORALES, quien en 1969 le presento a FAUSTO AVILA, del grupo de Genaro Vazquez Rojas y dias despues de ser secuetrado DONACIANO LUNA RADILLA, a quien liberaron en los primeros dias del mes de enero de 1971 mediante un rescate de $500,000.00 llegaron a su domicilio la Prof. SO= LIS MORALES Y SÁNTON MENDEZ BAILON llevandose '

# Clasificación

In [23]:
#Creacion entregable A

df2 = df[['nombre','texto_f']]
df2.to_csv( "drive/MyDrive/Reto2A.csv", index=False, encoding='utf-8-sig')

In [24]:
#Se da formato a los expedientes

df['class'] = 'Expediente'  
df['exp'] = df['exp'].apply(lambda x:re.sub("\n","",x))
exp = df[['nombre','exp','class']]
exp = exp.rename(columns={'nombre':'filename','exp':'label'})

In [25]:
exp.head()

,filename,label,class
0,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,Exp-11-240-74,Expediente
1,Ficheros_Detenidos_Desaparecidos_Morales_Lopez...,Exp.11-235-75,Expediente
2,"Ficheros_UAG_76-11-16_a_77-03-19_UAG,_fichas-1...",Exp.-100-10-1-77,Expediente
3,"Ficheros_UAG_73-10-29_a_73-12-07_UAG,_fichas-1...",,Expediente
4,Ficheros_Brigada_Campesina_de_Ajusticiamiento_...,Exp-100-10-16-4-72.,Expediente


In [26]:
from flair.data import Sentence
from flair.models import SequenceTagger
# Cargamos el tager que nos permite realizar la tarea de NER (Named entity recognition), el modelo
# a usar está disponible en (https://github.com/flairNLP/flair/)
tagger = SequenceTagger.load("flair/ner-spanish-large")

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2021-08-23 08:32:09,136 loading file /root/.flair/models/ner-spanish-large/045ad6c7dc21e0eb85935dce0544eec65f8c63c58412154df4dee7ff5f11665b.d4d3456316d2951bc100d060bd63a690b33af6d273adffa1b90df32328ed3257


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513 [00:00<?, ?B/s]

In [27]:
# REGEC para poder hacer la busqueda de los diferentes formatos de fechas
DATE_1 = "([0-9]|O|o){1,2}[(1|\||/|\\|I|!|¡|\(|\))]([0-9]|O|o){2}[(1|\||/|\\|I|!|¡|\(|\))]([0-9]|O|o){2,4}"
DATE_2 = "((^|\s|\/)((([0-3]|O|o){1}([0-9]|O|o){1})|([0-9]|O|o){1}))((\s)|(\.)|(\.\s)|(\.-)|(\.-\s)|(\-)|(\-\s)|(\_))(([efmajsond]{1}[a-z]{2,4})|([EFMAJSOND]{1}[a-z]{2,4})|([EFMAJSOND]{1}[A-Z]{2,3}))((\s)|(\.)|(\.-)|(\.-\s)|(\.\s)|(\-)|(\-\s)|(\_))(([1-9]|O|o){1}|([0-9]|O|o){2})(($)|(\s)|(\.)|(\.-)|(\-))"
DATE_3 = "(^|\s|\/)(([efmajsond]{1}[a-z]{2,4})|([EFMAJSOND]{1}[a-z]{2,4})|([EFMAJSOND]{1}[A-Z]{2,3}))((\s)|(\.)|(\.\s)|(\.-)|(\.-\s)|(\-)|(\-\s)|(\_))((([0-3]|O|o){1}([0-9]|O|o){1})|([0-9]|O|o){1})((\/)|(\s)|(\.)|(\.\s)|(\.-)|(\.-\s)|(\-)|(\-\s)|(\_)|(\&&)|(\&))(([1-9]|O|o){1}|([0-9]|O|o){2})(($)|(\s)|(\.)|(\.-)|(\-))"

In [28]:
def get_date(text):
  dates = []
  # Buscamos en las diferentes lineas del texto las fechas en sus 
  # diferentes formatos
  for line in text.split("\n"):
    resp1 = re.search(DATE_1, line)
    resp2 = re.search(DATE_2, line)
    resp3 = re.search(DATE_3, line)
    # En caso de hacer una detección y que este no sea un expediente, lo 
    # agregamos a nuestra lista de detecciones
    if resp1 is not None and "exp" not in resp1.group().lower():
      dates.append(resp1.group())
    if resp2 is not None and "exp" not in resp2.group().lower():
      dates.append(resp2.group())
    if resp3 is not None and "exp" not in resp3.group().lower():
      dates.append(resp3.group())
  return dates

In [29]:
def get_entity(text):
  # Generamos tres listas vacias donde iremos guardando las diferentes 
  # entidades detectadas
  LOC = []
  PER = []
  ORG = []
  sentence = Sentence(text.replace("-\n", "").replace("=\n", ""))
  tagger.predict(sentence)

  for entity in sentence.get_spans('ner'):
    if entity.tag == "PER" and entity.text not in PER:
      PER.append(entity.text)
    elif entity.tag == "ORG" and entity.text not in ORG:
      ORG.append(entity.text)
    elif entity.tag == "LOC" and entity.text not in LOC:
      LOC.append(entity.text)
      
  return LOC, PER, ORG

In [30]:
TAGS = "Fecha", "Lugar", "Persona", "Organización"
def create_df(nombre, dates, loc, per, org):
   # Agregamos las listas de las entidades detectadas a una nueva lista
  data = [dates, loc, per, org]
  # Ieramos sobre cada sub_lista de la nueva lista y vamos generando un nuevo 
  # arreglo con los datos
  data2 = [(nombre, entity, TAGS[n]) for n, sub_list in enumerate(data) for entity in sub_list]
  # Haciendo uso de la libreria numpy, este nuevo arreglo puede convertirse facilmente
  # en una matriz que posteriormente puede convertirse facilmente en un dataframe de pandas
  new_df = pd.DataFrame(np.asarray(data2), columns=["filename", "label", "class"])
  return new_df

In [31]:
from tqdm.notebook import tqdm
def flujo(dataframe,expedientes):
  # Por cada texto en nuestro dataframe de entrada, generamos las detecciones de
  # fechas y entidades 
  for i in tqdm(range(len(dataframe))):
    text = tmp_df.texto[i]
    dates = get_date(text)
    loc, per, org = get_entity(text)
    expedientes = expedientes.append(create_df(tmp_df.nombre[i], dates, loc, per, org))
  
  return expedientes

In [32]:
tmp_df = df.copy()

In [33]:
new_df = flujo(tmp_df,exp)

  0%|          | 0/10 [00:00<?, ?it/s]

In [34]:
new_df.sort_values(by=['filename'],inplace=True)

In [35]:
new_df.to_csv( "drive/MyDrive/Reto2B.csv", index=False, encoding='utf-8-sig')

In [36]:
new_df

,filename,label,class
0,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,Exp-11-240-74,Expediente
10,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,Escuela Superior de Medicina del I.P.N.,Organización
9,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,Asociación Mexicana de Médicos Residentes e Inte,Organización
8,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,Hospital Civil de Oaxaca,Organización
7,Ficheros_ACNR_militantes_Olea_Castaneyra_Rafel...,SÁNTON MENDEZ BAILON,Persona
...,...,...,...
0,"Ficheros_Union_Campesina_Independiente,_fichas...",2-Abril-77.,Fecha
9,"Ficheros_Union_Campesina_Independiente,_fichas...",Exp-100-19-1-77.-,Expediente
11,"Ficheros_Union_Campesina_Independiente,_fichas...",Univń,Organización
4,"Ficheros_Union_Campesina_Independiente,_fichas...",Mpio.de San José Acateno,Lugar
